In [1]:
import pandas as pd
import numpy as np
import nltk
import math
import time
import operator
import string
import csv
import nltk
from sklearn.cluster import KMeans
from math import pi
from sklearn.metrics import mean_squared_error
from nltk.corpus import wordnet, stopwords, sentiwordnet as swn
from nltk.tokenize import sent_tokenize, word_tokenize, WhitespaceTokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
f = open('train_data.csv','r', errors='ignore')
train_data = pd.read_csv(f,
                header=0, sep=',',
                parse_dates=['date', 'release_date', 'update_date'],
                infer_datetime_format=True)
length_train = len(train_data.review_text)

f1 = open('test_data.csv','r', errors='ignore')
test_data = pd.read_csv(f1,
                header=0, sep=',',
                parse_dates=['date', 'release_date', 'update_date'],
                infer_datetime_format=True)
length_test = len(test_data.review_text)

In [4]:
train_review = pd.Series.tolist(train_data.review_text)
train_rating = pd.Series.tolist(train_data.star_rating)
# train_asin = pd.Series.tolist(train_data.asin)
# train_app_purch = pd.Series.tolist(train_data.in_app_purchase)


# train_asin1 = list(set(train_asin))
# for i in range(len(train_asin)):
#     for j in range(len(train_asin1)):
#         if train_asin[i] == train_asin1[j]:
#             train_asin[i] = j;

# total_pack = zip(train_asin,train_app_purch,train_rating)
# train_model2 = [None]*5
train_model = [None]*5
for i in range(5):
    train_model[i] = [str(tmp) for tmp, score in zip(train_review, train_rating) if score == i+1]
#     train_model2[i] = [[asin,app] for asin,app,score in total_pack if score == i+1]

In [6]:
most_word = [None]*5
feature = [None]*5
star_words = [None]*5
#a = []
test_review = pd.Series.tolist(test_data.review_text)
test_review = [str(tmp) for tmp in test_review]
voc_model = TfidfVectorizer(decode_error='ignore', max_df=0.8, min_df=0.01, stop_words='english')
for i in range(5):
    feature[i] = voc_model.fit_transform(train_model[i])
    most_word[i] = voc_model.get_feature_names()
# for i in range(5):
#     for t,j in nltk.pos_tag(most_word[i]):
#         if j in ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']: 
#             a.append(t)
#     star_words[i] = a
            
most_test_words = [None]*length_test

In [ ]:
# start = time.time()
# star_rate_predict = [None]*length_test
# for m,sentence in enumerate(test_review):
#     test_feature_words = []
#     rate = 0
#     for i,j in nltk.pos_tag(sentence):
#         if j in ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']: 
#             test_feature_words.append(i)
#     for words in test_feature_words:
#         for i in range(5):
#             if words in star_words[i]:
#                 rate = rate + i - 2
#     star_rate_predict[m] = rate
# end = time.time()
# print(end-start)

# test_view1 = [None]*
# for i in range(len(test_review)):
# #     if test_data1[i] == None:
# #         test_data1[i] = the
#     test_review1[i] = "".join([ch for ch in test_data1[i] if ch not in set(string.punctuation)])
# #for i in range(5):
# #     for j in range(len(most_word[i])):
# test_review1 = [None]*len(test_review)       
# for i in range(len(test_review)):
#     test_review1[i] = ''.join(tmp for tmp in test_review if tmp not in set(string.punctuation))  
    

In [6]:
start = time.time()
star_rate_predict = [None]*length_test
for m,sentence in enumerate(test_review):
    rate = 0
    words = word_tokenize(sentence)
    for w in words:
        s = 0
        rate_w = 0
        for i in range(5):
            if w.lower() in most_word[i]:
                s = s + 1
                rate_w = rate_w + i - 1
        if s != 0:
            rate_w = rate_w / s
        rate = rate + rate_w
    star_rate_predict[m] = rate
end = time.time()
print(end-start)

93.97746205329895


In [7]:
# kmeans = KMeans(n_clusters = 5)
star = [[tmp] for tmp in star_rate_predict]
# kmeans.fit(star)
# centroids = kmeans.cluster_centers_
# centroids1 = sorted(centroids)
# print(centroids1)

In [17]:
standard = [[-1],[-0.5],[0],[1],[2]]
distance = np.zeros((100000,5))
for i in range(len(star_rate_predict)):
    for j in range(5):
        distance[i][j] = mean_squared_error(standard[j], star[i])
        if distance[i][j] == 0:
            distance[i][j] = 0.001

In [18]:
prob = np.zeros((100000,5))
distance1 = 1/distance
for i in range(100000):
    for j in range(5):
        prob[i][j] = (distance1[i][j]/sum(distance1[i]))
prob

array([[ 0.01701677,  0.02316172,  0.03335287,  0.09264686,  0.83382178],
       [ 0.15588272,  0.16914357,  0.18417145,  0.22092222,  0.26988005],
       [ 0.04152151,  0.05423218,  0.07381602,  0.16608606,  0.66434422],
       ..., 
       [ 0.1371781 ,  0.15379136,  0.17361603,  0.22676379,  0.30865072],
       [ 0.15768132,  0.17054812,  0.18505655,  0.22023259,  0.26648143],
       [ 0.11704977,  0.13575003,  0.15931774,  0.22941755,  0.35846492]])

In [20]:
prob1 = prob.astype('str')
review_id = pd.Series.tolist(test_data.review_id)
review_id = np.asarray([[str(tmp)] for tmp in review_id])
result = np.append(review_id, prob1, 1)
title = np.asarray([['review_id','1-star','2-star','3-star','4-star','5-star']])
result = np.append(title,result,0)
#result = [np.array(tmp) for tmp in result]
result = result.tolist()
result = np.asarray(result)
np.savetxt("result1.csv",result,fmt='%s,%s,%s,%s,%s,%s')